In [2]:
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

np.unique(iris_y)

array([0, 1, 2])

In [18]:
# Split iris data in train and test data
# A random permutation, to split the data randomly
np.random.seed(0)

indices = np.random.permutation(len(iris_X))
iris_X_train = iris_X[indices[:-10]]
iris_y_train = iris_y[indices[:-10]]
iris_X_test  = iris_X[indices[-10:]]
iris_y_test  = iris_y[indices[-10:]]

# Create and fit a nearest-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(iris_X_train, iris_y_train)

# Predict based on model fit
p = knn.predict(iris_X_test)

print('Prediction: ', p)
print('Actual:     ', iris_y_test)

Prediction:  [1 2 1 0 0 0 2 1 2 0]
Actual:      [1 1 1 0 0 0 2 1 2 0]


In [22]:
# Linear model

# Linear regression
diabetes = datasets.load_diabetes()
diabetes_X_train = diabetes.data[:-20]
diabetes_X_test  = diabetes.data[-20:]
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test  = diabetes.target[-20:]

from sklearn import linear_model
regr = linear_model.LinearRegression()

regr.fit(diabetes_X_train, diabetes_y_train)
print(regr.coef_)

# m=Mean square error
m = np.mean((regr.predict(diabetes_X_test)-diabetes_y_test)**2)
print('Mean square error: ', m)

# Variance score: 1 is perfect prediction and 0 means 
# that there is no linear relationship between X and y.
vs = regr.score(diabetes_X_test, diabetes_y_test) 
print('Variance score: ', vs)

[  3.03499549e-01  -2.37639315e+02   5.10530605e+02   3.27736980e+02
  -8.14131709e+02   4.92814588e+02   1.02848452e+02   1.84606489e+02
   7.43519617e+02   7.60951722e+01]


0.58507530226905724